In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.regularizers import l2
from sklearn.model_selection import train_test_split
from collections import Counter

2025-02-22 22:28:39.023225: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-22 22:28:39.132923: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-22 22:28:39.190356: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-22 22:28:39.213161: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-22 22:28:39.305784: I tensorflow/core/platform/cpu_feature_guar

In [2]:
reviews = pd.read_csv('reviews.txt', header=None)
labels = pd.read_csv('labels.txt', header=None)

In [3]:
def preprocess_text(text):
    text = text.lower()
    text = ''.join([c for c in text if c.isalpha() or c.isspace()])
    return text

total_counts = Counter()
for _, row in reviews.iterrows():
    processed_text = preprocess_text(row[0])
    total_counts.update(processed_text.split())

In [4]:
vocab = [word for word, _ in total_counts.most_common(10000)]
print(vocab[:60])

['the', 'and', 'a', 'of', 'to', 'is', 'br', 'it', 'in', 'i', 'this', 'that', 's', 'was', 'as', 'for', 'with', 'movie', 'but', 'film', 'you', 'on', 't', 'not', 'he', 'are', 'his', 'have', 'be', 'one', 'all', 'at', 'they', 'by', 'an', 'who', 'so', 'from', 'like', 'there', 'her', 'or', 'just', 'about', 'out', 'if', 'has', 'what', 'some', 'good', 'can', 'more', 'she', 'when', 'very', 'up', 'time', 'no', 'even', 'my']


In [5]:
print(vocab[-1], ': ', total_counts[vocab[-1]])

chalk :  30


In [6]:
word2idx = {word: i for i, word in enumerate(vocab)}

In [7]:
def text_to_vector(text):
    vector = np.zeros(len(vocab), dtype=np.float32)
    processed_text = preprocess_text(text)
    for word in processed_text.split():
        if word in word2idx:
            vector[word2idx[word]] += 1
    return vector

In [8]:
text_to_vector('The tea is for a party to celebrate '
               'the movie so she has no time for a cake')[:65]

array([2., 0., 2., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 0.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)

In [9]:
labels = (labels[0] == 'positive').astype(np.int32)
vectors = np.array([text_to_vector(text) for text in reviews[0]])

In [10]:
vectors[:5, :23]

array([[ 9.,  1.,  4.,  4.,  6.,  4.,  0.,  2.,  2.,  5.,  0.,  4.,  1.,
         0.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
       [ 4.,  1.,  7.,  3.,  1.,  2.,  0.,  4.,  0.,  0.,  0.,  1.,  2.,
         0.,  0.,  1.,  3.,  0.,  0.,  0.,  1.,  1.,  0.],
       [24.,  4., 17.,  5., 20.,  2.,  8.,  8.,  2.,  1.,  1.,  2.,  8.,
         0.,  5.,  5.,  4.,  0.,  2.,  1.,  4., 10.,  4.],
       [53.,  0., 22., 23., 13., 14.,  8., 10.,  8., 12.,  9.,  4., 11.,
         2., 11.,  5., 11.,  0.,  5.,  3.,  0.,  3.,  7.],
       [10.,  4.,  6.,  2.,  2.,  5.,  0.,  1.,  2.,  3.,  1.,  0.,  0.,
         0.,  3.,  1.,  0.,  1.,  0.,  0.,  0.,  2.,  0.]], dtype=float32)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(
    vectors, labels, test_size=0.1, stratify=labels, random_state=42
)

y_train = tf.keras.utils.to_categorical(y_train, num_classes=2)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=2)

In [12]:
y_train

array([[1., 0.],
       [1., 0.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [0., 1.]])

In [13]:
def build_model():
    model = Sequential([
        Dense(256, activation='relu', input_shape=(10000,)),
        Dropout(0.5),
        Dense(128, activation='relu', kernel_regularizer=l2(0.001)),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dense(2, activation='softmax')
    ])
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy', 
                tf.keras.metrics.Precision(name='precision'),
                tf.keras.metrics.Recall(name='recall')]
    )
    return model

In [14]:
model = build_model()
model.summary()

/home/ghaidaa/anaconda3/envs/tf_env_v2/lib/python3.9/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1740256158.937423  108009 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1740256159.079212  108009 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1740256159.080378  108009 cuda

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 256)            │     2,560,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,601,538 (9.92 MB)

 Trainable params: 2,601,538 (9.92 MB)

 Non-trainable params: 0 (0.00 B)

In [15]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=5, restore_best_weights=True
)

history = model.fit(
    X_train, y_train,
    epochs=10,
    batch_size=256,
    validation_split=0.2,
    callbacks=[early_stopping],
    verbose=1
)

Epoch 1/10


I0000 00:00:1740256162.735922  108577 service.cc:146] XLA service 0x7f1ad8004070 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1740256162.735974  108577 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Laptop GPU, Compute Capability 8.6
2025-02-22 22:29:22.785658: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-02-22 22:29:22.992892: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907


27/71 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5843 - loss: 0.8519 - precision: 0.5843 - recall: 0.5843

I0000 00:00:1740256165.670109  108577 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


71/71 ━━━━━━━━━━━━━━━━━━━━ 15s 151ms/step - accuracy: 0.6735 - loss: 0.7460 - precision: 0.6735 - recall: 0.6735 - val_accuracy: 0.8682 - val_loss: 0.4384 - val_precision: 0.8682 - val_recall: 0.8682
Epoch 2/10
71/71 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9051 - loss: 0.3430 - precision: 0.9051 - recall: 0.9051 - val_accuracy: 0.8784 - val_loss: 0.3789 - val_precision: 0.8784 - val_recall: 0.8784
Epoch 3/10
71/71 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9386 - loss: 0.2366 - precision: 0.9386 - recall: 0.9386 - val_accuracy: 0.8900 - val_loss: 0.3569 - val_precision: 0.8900 - val_recall: 0.8900
Epoch 4/10
71/71 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9660 - loss: 0.1480 - precision: 0.9660 - recall: 0.9660 - val_accuracy: 0.8900 - val_loss: 0.3710 - val_precision: 0.8900 - val_recall: 0.8900
Epoch 5/10
71/71 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9811 - loss: 0.1043 - precision: 0.9811 - recall: 0.9811 - val_accuracy: 0.8833 - val_loss: 0.4536 - val_precisi

In [16]:
import pickle
model.save('sentiment_analysis_model.h5')
with open('word2idx.pkl', 'wb') as f:
    pickle.dump(word2idx, f)
model = tf.keras.models.load_model('sentiment_analysis_model.h5')
with open('word2idx.pkl', 'rb') as f:
    word2idx = pickle.load(f)

print("Model and tokenizer data successfully loaded!")

Model and tokenizer data successfully loaded!


In [17]:
test_loss, test_acc, test_precision, test_recall = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_acc:.4f}")
print(f"Test precision: {test_precision:.4f}")
print(f"Test recall: {test_recall:.4f}")

79/79 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step - accuracy: 0.8747 - loss: 0.3982 - precision: 0.8747 - recall: 0.8747

Test accuracy: 0.8832
Test precision: 0.8832
Test recall: 0.8832


In [21]:
def test_sentence(sentence):
    vector = text_to_vector(sentence)
    prediction = model.predict(np.array([vector]))
    positive_prob = prediction[0][1]
    confidence = max(prediction[0]) * 100
    
    print(f"Sentence: {sentence}")
    print(f"Prediction: {'Positive' if positive_prob > 0.5 else 'Negative'}")
    print(f"Confidence: {confidence:.1f}%")
    print(f"Positive probability: {positive_prob:.3f}")

test_sentence("This movie was absolutely fantastic!")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
Sentence: This movie was absolutely fantastic!
Prediction: Positive
Confidence: 76.2%
Positive probability: 0.762


## Test

In [22]:
sentence = "Moonlight is by far the best movie of 2016."
test_sentence(sentence)

sentence = "It's amazing anyone could be talented enough to make something this spectacularly awful"
test_sentence(sentence)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Sentence: Moonlight is by far the best movie of 2016.
Prediction: Positive
Confidence: 70.7%
Positive probability: 0.707
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Sentence: It's amazing anyone could be talented enough to make something this spectacularly awful
Prediction: Negative
Confidence: 92.3%
Positive probability: 0.077
